In [1]:
import pandas as pd
import gurobipy as gb
from gurobipy import GRB
import numpy as np
import warnings
import time
import asyncio
from matplotlib import pyplot as plt
import geopandas as gpd

# Load data
df_main = pd.read_excel("C:\\Users\\smmashai\\Downloads\\Thesis\\master-shaik-main\\classic-model\\SimData\\Nodes old RTF.xlsx")

Nodes = df_main.iloc[:, [0, 2, 3]]
Supply = df_main.iloc[:, [4, 5, 6, 7, 8, 9, 10]]
Demand = df_main.iloc[:, [11, 12, 13, 14, 15, 16, 17]]

Materials = ["Mixed", "Foam", "Briq", "pyrOil", "ANL", "P-TOL", "NaphtaSub"]
tech = ["CF", "RTF", "CPF", "DPF"]

yield_array = [[-1, 0.01, 0, 0, 0, 0, 0],
                [0, -1, 0.8, 0, 0, 0, 0],
                [0, 0, -1, 0.635, 0, 0, 0],
                [0, 0, 0, -1,  0.47, 0.136, 0.37]]

actual_marketprice = [0, 0, 0, 0, 1495, 1500, 380]
m = 2
marketprice = [m * price for price in actual_marketprice]
final_og = 5
opt_og = 0.15
facility_capacity_list = [[100,80,60,50,40], [180,160,130,90,80], [180,160,130,90,80], [120,110,80,60,50]]
#facility_capacity_list = [[960, 760, 560, 360, 160], [70, 55, 40, 25, 10], [60, 50, 40, 30, 20], [40, 32, 25, 17, 10]]
#facility_capacity_list = [[2000, 1650, 1300, 950, 600], [100, 80, 60, 40, 20], [180, 160, 140, 120, 100], [140, 120, 100, 80, 60]]

# Reference costs and capacities
refFixedCF = 0#1.45  # MM euro
refFixedRTF = 2.88#12.8  # MM euro
refFixedCPF = 52.5  # MM euro
refFixedDPF = 47.7  # MM euro
refCapacityCF = 100  # ton/day
refCapacityRTF = 100  # ton/day
refCapacityCPF = 300  # ton/day
refCapacityDPF = 192  # ton/day
capexScalingFactor = 0.6
rate = 0.1
period = 10

# Overhead costs
overheadCF = 0.267 * 0.01  # MM euro/year
overheadRTF = 0.817  # MM euro/year
overheadCPF = 3.66  # MM euro/year
overheadDPF = 3.52  # MM euro/year
workingDays = 330

# Function to calculate facility costs
def calc_facility_cost(k):
    tot_cost_CF = 0 #refFixedCF * ((facility_capacity_list[0][k] / refCapacityCF) ** capexScalingFactor) * (10 ** 6)
    tot_cost_RTF = refFixedRTF * ((facility_capacity_list[1][k] / refCapacityRTF) ** capexScalingFactor) * (10 ** 6)
    tot_cost_CPF = refFixedCPF * ((facility_capacity_list[2][k] / refCapacityCPF) ** capexScalingFactor) * (10 ** 6)
    tot_cost_DPF = refFixedDPF * ((facility_capacity_list[3][k] / refCapacityDPF) ** capexScalingFactor) * (10 ** 6)
    Annual_cost_CF = rate * tot_cost_CF / (1 - (1 + rate) ** (-period))
    Annual_cost_RTF = rate * tot_cost_RTF / (1 - (1 + rate) ** (-period))
    Annual_cost_CPF = rate * tot_cost_CPF / (1 - (1 + rate) ** (-period))
    Annual_cost_DPF = rate * tot_cost_DPF / (1 - (1 + rate) ** (-period))
    return [Annual_cost_CF, Annual_cost_RTF, Annual_cost_CPF, Annual_cost_DPF]

facility_total_cost = [calc_facility_cost(k) for k in range(len(facility_capacity_list[0]))]
facility_total_cost = list(map(list, zip(*facility_total_cost)))

facility_var_costs = [0.15, 46, 68, 102]  # Updated variable costs

# Calculate fixed operating costs (fOPEX)
def calc_fixed_operating_cost(k):
    fOPEX_CF = overheadCF * ((facility_capacity_list[0][k] / refCapacityCF) ** capexScalingFactor) * (10 ** 6) 
    fOPEX_RTF = overheadRTF * ((facility_capacity_list[1][k] / refCapacityRTF) ** capexScalingFactor) * (10 ** 6) 
    fOPEX_CPF = overheadCPF * ((facility_capacity_list[2][k] / refCapacityCPF) ** capexScalingFactor) * (10 ** 6) 
    fOPEX_DPF = overheadDPF * ((facility_capacity_list[3][k] / refCapacityDPF) ** capexScalingFactor) * (10 ** 6) 
    return [fOPEX_CF, fOPEX_RTF, fOPEX_CPF, fOPEX_DPF]

fixed_operating_costs = [calc_fixed_operating_cost(k) for k in range(len(facility_capacity_list[0]))]
fixed_operating_costs = list(map(list, zip(*fixed_operating_costs)))
transport_costs = [0.14124335003280253, 0.2600782146695747, 0.057932422317647767, 0.03245140669834265, 0.025961125358674116, 0.025961125358674116, 0.025961125358674116]

# Function to calculate distances using Haversine formula
def distanceHaversine(city, lat, lon):
    df_dist = pd.DataFrame([[0] * (len(city) + 1)] * (len(city) + 1))
    city_names = city
    lat_df = lat
    lon_df = lon
    radius = 6371

    for i in range(len(city)):
        df_dist.iloc[i + 1, 0] = city_names.iloc[i]
        df_dist.iloc[0, i + 1] = city_names.iloc[i]
        for j in range(len(city)):
            lat1 = lat_df.iloc[i]
            lon1 = lon_df.iloc[i]
            lat2 = lat_df.iloc[j]
            lon2 = lon_df.iloc[j]
            delta_lat = np.radians(lat2 - lat1)
            delta_lon = np.radians(lon2 - lon1)

            a = (np.sin(delta_lat / 2)) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * (np.sin(delta_lon / 2)) ** 2
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
            d = radius * c
            df_dist.iloc[i + 1, j + 1] = d
    return df_dist

warnings.simplefilter(action='ignore', category=FutureWarning)

df_dist_withcitynames = distanceHaversine(Nodes.iloc[:, 0], Nodes.iloc[:, 1], Nodes.iloc[:, 2])
df_distances = df_dist_withcitynames.iloc[1:, 1:].apply(pd.to_numeric)

multipliers = {0: 0.2, 1: 2, 2: 2, 3: 2}
num_nodes = len(Nodes)
num_mat = len(Materials)
num_techs = len(tech)
num_cap = len(facility_capacity_list[0])



start_time_lb = time.time()
full_problem_model_lb = gb.Model("FullProblem lb")
full_problem_model_lb.setParam('MIPGap', opt_og)



flow = full_problem_model_lb.addVars(((i, j, m) for i in range(num_nodes) for j in range(num_nodes) for m in range(num_mat)), vtype=gb.GRB.CONTINUOUS)
demand_node = full_problem_model_lb.addVars(num_nodes, num_mat, vtype=GRB.CONTINUOUS)
supply_node = full_problem_model_lb.addVars(num_nodes, num_mat, vtype=GRB.CONTINUOUS)
tech_capacity = full_problem_model_lb.addVars(num_nodes, num_techs, vtype=GRB.CONTINUOUS)
operating_cost = full_problem_model_lb.addVar(vtype=GRB.CONTINUOUS)
fixed_operating_cost = full_problem_model_lb.addVar(vtype=GRB.CONTINUOUS)
capital_cost = full_problem_model_lb.addVar(vtype=GRB.CONTINUOUS)
transport_cost = full_problem_model_lb.addVar(vtype=GRB.CONTINUOUS)
tech_decision = full_problem_model_lb.addVars(num_nodes, num_techs, num_cap, vtype=GRB.BINARY)

for i in range(num_nodes):
    for t in range(num_techs):
        full_problem_model_lb.addConstr(tech_capacity[i, t] >= 0)
        full_problem_model_lb.addConstr(tech_capacity[i, t] <= gb.quicksum(tech_decision[i,t,c]*facility_capacity_list[t][c] for c in range(num_cap) ))
        full_problem_model_lb.addConstr(1 >= gb.quicksum(tech_decision[i,t,c] for c in range(num_cap) ))

# Add objective function: maximize social welfare
obj = full_problem_model_lb.addVar(vtype=gb.GRB.CONTINUOUS)

# Constraints for supply, demand, and flow conservation
for i in range(num_nodes):
    for m in range(num_mat):
        full_problem_model_lb.addConstr(supply_node[i, m] <= Supply.iloc[i, m])
        full_problem_model_lb.addConstr(demand_node[i, m] <= Demand.iloc[i, m])
        full_problem_model_lb.addConstr(
            supply_node[i, m] + gb.quicksum(flow[j, i, m] for j in range(num_nodes) ) + gb.quicksum(yield_array[t][m]*tech_capacity[i,t] for t in range(num_techs)) == demand_node[i, m] + gb.quicksum(flow[i, j, m] for j in range(num_nodes)))
#full_problem_model_lb.addConstr(gb.quicksum(supply_node[i, 0] for i in range(num_nodes)) >= 0.8*gb.quicksum(Supply.iloc[i, 0] for i in range(num_nodes)))

# Constraints for transportation cost, operating cost, and capital cost
full_problem_model_lb.addConstr(
    transport_cost == gb.quicksum(transport_costs[m] * df_distances.iloc[i,j]*flow[i, j, m] for i in range(num_nodes) 
                                for j in range(num_nodes) for m in range(num_mat) ))
full_problem_model_lb.addConstr(
    operating_cost == gb.quicksum(facility_var_costs[t]*tech_capacity[i, t] for i in range(num_nodes) #variable costs per day
                                    for t in range(num_techs)))
full_problem_model_lb.addConstr(
        fixed_operating_cost == gb.quicksum(tech_decision[i,t,c]*fixed_operating_costs[t][c] for i in range(num_nodes) 
                                    for t in range(num_techs) for c in range(num_cap)))
full_problem_model_lb.addConstr(
    capital_cost == gb.quicksum(tech_decision[i,t,c]*facility_total_cost[t][c]  for i in range(num_nodes) 
                                    for t in range(num_techs) for c in range(num_cap)))#capital cost per year

full_problem_model_lb.addConstr(
    obj == workingDays*gb.quicksum(marketprice[m]*demand_node[i,m] for i in range(num_nodes) for m in range(num_mat)) - 2*workingDays*transport_cost - workingDays*operating_cost-fixed_operating_cost-capital_cost)

full_problem_model_lb.setObjective(obj, gb.GRB.MAXIMIZE)

# Optimize the model
full_problem_model_lb.optimize()
end_time_lb = time.time()
revenue = 0

# Retrieve the values of demand_node variables
for i in range(num_nodes):
    for m in range(num_mat):
        revenue += marketprice[m] * demand_node[i, m].X

# Multiply by workingDays as per the objective function definition
revenue *= workingDays
# Sum tech capacities across all nodes for each technology
tech_capacity_sums = {t: sum(tech_capacity[i, t].X for i in range(num_nodes)) for t in range(num_techs)}

# Print the summed tech capacities for each technology
for t in range(num_techs):
    print(f"Total capacity for technology {t}: {tech_capacity_sums[t]}")

# Sum demands across all nodes for each material
demand_sums = {m: sum(demand_node[i, m].X for i in range(num_nodes)) for m in range(num_mat)}

# Print the summed demands for each material
for m in range(num_mat):
    print(f"Total demand for material {m}: {demand_sums[m]}")
a = operating_cost.X
d = fixed_operating_cost.X
b =transport_cost.X
c= capital_cost.X
# Print the revenue
print(f"Revenue: {revenue}")
print(f"Operating cost: {a}")
print(f"transport cost: {b}")
print(f"capital cost: {c}")
print(f"fixed Operating cost: {d}")
last_tech_capacity_lb = {(i, t): tech_capacity[i, t].X for i in range(num_nodes) for t in range(num_techs)}
last_tech_decision_lb = {(i, t, c): tech_decision[i, t, c].X for i in range(num_nodes) for t in range(num_techs) for c in range(num_cap)}


# Create an empty dictionary to store tech capacities
tech_capacity_list = {f"Tech_{t}_capacity": [] for t in range(num_techs)}


# Sum tech capacities across all nodes for each technology
actual_capacity_used = {t: sum(tech_capacity[i, t].X for i in range(num_nodes)) for t in range(num_techs)}

# Print the summed actual capacities used for each technology
for t in range(num_techs):
    print(f"Actual capacity used for technology {t}: {actual_capacity_used[t]}")

# Calculate and print the built capacity and percentage facility usage
built_capacity = {t: sum(tech_decision[i, t, c].X * facility_capacity_list[t][c] for i in range(num_nodes) for c in range(num_cap)) for t in range(num_techs)}

for t in range(num_techs):
    usage_percentage = (actual_capacity_used[t] / built_capacity[t]) * 100 if built_capacity[t] > 0 else 0
    print(f"Built capacity for technology {t}: {built_capacity[t]}")
    print(f"Percentage facility usage for technology {t}: {usage_percentage:.2f}%")

  
    

C:\Users\smmashai\AppData\Local\Temp\ipykernel_28220\4173457352.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-10
Set parameter MIPGap to value 0.15
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12600, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4889 rows, 158957 columns and 475829 nonzeros
Model fingerprint: 0x4b7c3371
Variable types: 155997 continuous, 2960 integer (2960 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 1e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 2668 rows and 2076 columns
Presolve time: 0.42s
Presolved: 2221 rows, 156881 columns, 469580 nonzeros
Variable types: 153921 continuous, 2960 integer (2960 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log on